### Import Libraries

In [1]:
import os
import pickle
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Load Trained Model

In [2]:
def load_artifacts():
    '''
    Loads the saved machine learning model and scaler from disk.
    
    Returns:
        model (object): The pre-trained machine learning model.
        scaler (object): The pre-fitted scaler loaded .
    '''
    # Define the path for the saved model file
    model_path = os.path.join("..","model",'saved_model.pkl')
    model = pickle.load(open(model_path,'rb'))

    # Define the path for the saved scaler file
    scaler_path = os.path.join("..","artifacts",'scaler.pkl')
    scaler = pickle.load(open(scaler_path,'rb'))

    return model, scaler

model, scaler = load_artifacts()

### Model Prediction

In [3]:
def predict_price(inputs: list) -> float:
    """
    Predict the car price based on user input.
    
    Args:
        inputs (list): List of form values [Brand, Year, KMs Driven, Ownership, Fuel Type]
    
    Returns:
        float: Predicted price of the car
    """
    # Initialize feature vector
    feature_names = model.feature_names_in_
    input_vector = np.zeros(feature_names.shape)

    # Extract form inputs
    car_brand, year, kms_driven, owner_type, fuel_type = inputs

    # Calculate car age
    current_year = datetime.date.today().year
    car_age = current_year - int(year)

    # Scale numerical features (kms driven and age)
    input_vector[:2] = scaler.transform(np.expand_dims([float(kms_driven), car_age], axis=0))

    # One-hot encode categorical features if they exist in training features
    feature_mapping = {
        f'car_{car_brand}': car_brand,
        f'ownership_{owner_type}': owner_type,
        f'fuel_type_{fuel_type}': fuel_type
    }

    for feature_key in feature_mapping:
        if feature_key in feature_names:
            feature_idx = np.where(feature_names == feature_key)[0][0]
            input_vector[feature_idx] = 1

    # Predict log price and convert back using exponential
    predicted_log_price = model.predict(input_vector[np.newaxis, ...])
    predicted_price = np.exp(predicted_log_price)

    return predicted_price[0]


### Prompt User Input

In [4]:
def take_user_inputs()-> list:
    '''
    This function prompts the user to enter details about the car, such as:
    - car brand (capitalized)
    - year of the car
    - kilometers driven
    - owner type (mapped from numeric input)
    - fuel type (mapped from numeric input)

    Returns:
        list: A list containing the car brand, year, kilometers driven, owner type, and fuel type.
    '''
    
    car_brand = input("Enter the car brand : ").capitalize()
    year = input("Enter the car year : ")
    kms_driven = input("Enter the number of kilometers driven : ")
    owner_type = input("Enter the owner type -\npress 1 for 'First Owner'\npress 2 for 'Second owner'\npress 3 for 'Third Owner'\n: ")
    fuel_type = input("Enter the fuel type -\npress 1 for 'Petrol'\npress 2 for 'Diesel'\n:")
    print()
    inputs = [car_brand, year, kms_driven, owner_type, fuel_type]
    return inputs

In [5]:
def validate_inputs(inputs:list)->bool:
    """
    Validates the user inputs for car details

    Args:
        inputs (list): A list containing the car details (car_brand, year, kms_driven, owner_type, fuel_type).

    Returns:
        bool: `True` if there are errors in the inputs, `False` otherwise.
    """
    
    car_brand, year, kms_driven, owner_type, fuel_type = inputs
    error = False
    curr_year = datetime.date.today().year
    
    # Validate car brand
    feature_names = model.feature_names_in_
    if not any(car_brand in car_name for car_name in feature_names):
        print(car_brand,": car name not found")
        error = 1

    # Validate the year input
    if year.isdigit() and 1994 < int(year) <  curr_year:
        year = int(year)
    else:
        print(f"{year} : The year should be in between 1994 -- {curr_year}")
        error = 1

    # Validate kilometers driven input
    if  kms_driven.isdigit():
        kms_driven = int(kms_driven)
    else:
        print(kms_driven,": invalid kms type")
        error = 1

    # Validate owner type input
    if owner_type.isdigit() and int(owner_type)>0 :
        owner_type = int(owner_type) if int(owner_type)<4 else 3
    else:
        print(owner_type,": invalid format")
        error = 1

    # Validate fuel type input
    if fuel_type.isdigit() and int(fuel_type) in [1,2]:
        fuel_type = int(owner_type)
    else:
        print(fuel_type,": invalid format")
        error = 1

    # Return the error status
    return error

In [6]:
def main():
    """
    Main entry point of the program that orchestrates user input, validation, 
    and price prediction.

    This function:
    1. 
    2. 
    3. If inputs are valid, it predicts the car price using a machine learning model.
    4. If inputs are invalid, prompts the user to try again.
    """
    # Takes user inputs for car details.
    inputs = take_user_inputs()

    # Validates the inputs.
    error = validate_inputs(inputs)

    # If no error, predict the car price
    if not error :
        price = predict_price(inputs)
        print(f'Estimated Price : {int(price):,} Rupees')
    else : 
        print("\nPlease Try Again!!!")

In [7]:
if __name__ == "__main__":
    main()

Enter the car brand :  Swift
Enter the car year :  2016
Enter the number of kilometers driven :  110500
Enter the owner type -
press 1 for 'First Owner'
press 2 for 'Second owner'
press 3 for 'Third Owner'
:  2
Enter the fuel type -
press 1 for 'Petrol'
press 2 for 'Diesel'
: 1



Estimated Price : 461,776 Rupees


-----